In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import joblib
from sklearn import metrics
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression



In [ ]:
df = pd.read_pickle('../../data.scenarioJunctionExo.pkl.gz')


In [ ]:
def pipeline_save(df):
    X = df.drop('winner', axis=1)
    y = df['winner']    
    #categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c=df.select_dtypes(include=['object']).drop(['winner'], axis=1).columns
    preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), c)])
    rf = Pipeline(steps=[('preprocessor', preprocessor),('classifier', RandomForestClassifier())])
    rf.fit(X, y)
    return(rf)


In [ ]:
def pipelineClassifier(df):   
    X = df.drop('winner', axis=1)
    y = df['winner']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c=df.select_dtypes(include=['object']).drop([['winner', 'meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed']], axis=1,errors="ignore").columns
    preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, c)])
    classifiers = [
        KNeighborsClassifier(3),
        SVC(kernel="rbf", C=0.025, probability=True),
        NuSVC(probability=True),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        AdaBoostClassifier(),
        GradientBoostingClassifier()
        ]
    for classifier in classifiers:
        pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', classifier)])
        pipe.fit(X_train, y_train)
        y_pred=pipe.predict(X_test)
        print(classifier)
        print("model score: %.3f" % pipe.score(X_test, y_test))
        print("model accuracy:",metrics.accuracy_score(y_test,y_pred))
        #print("model f1 score:",metrics.f1_score(y_test,y_pred))

In [ ]:
def pipelineClassifierCross(df):   
    X = df.drop('winner', axis=1)
    y = df['winner']
    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    c=df.select_dtypes(include=['object']).drop(['winner'], axis=1).columns
    preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, c)])
    classifiers = [
        KNeighborsClassifier(3),
        #SVC(kernel="rbf", C=0.025, probability=True),
        NuSVC(probability=True),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        AdaBoostClassifier(),
        GradientBoostingClassifier()
        ]
    for classifier in classifiers:
        pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', classifier)])
        scores = cross_val_score(pipe, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
        print(classifier.__class__.__name__)
        print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))
     







In [ ]:
pipelineClassifierCross(df)

In [ ]:
model=joblib.load('model.joblib')